In [ ]:
!wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/images/Positive_tensors.zip

--2024-08-02 11:58:26--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/images/Positive_tensors.zip
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2598656062 (2.4G) [application/zip]
Saving to: ‘Positive_tensors.zip’

Positive_tensors.zi 100%[===================>]   2.42G  22.6MB/s    in 1m 50s  

2024-08-02 12:00:17 (22.6 MB/s) - ‘Positive_tensors.zip’ saved [2598656062/2598656062]



In [ ]:
!unzip -q Positive_tensors.zip

In [ ]:
! wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/images/Negative_tensors.zip

--2024-08-02 12:02:41--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/images/Negative_tensors.zip
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2111408108 (2.0G) [application/zip]
Saving to: ‘Negative_tensors.zip’

Negative_tensors.zi 100%[===================>]   1.97G  30.0MB/s    in 58s     

2024-08-02 12:03:40 (34.6 MB/s) - ‘Negative_tensors.zip’ saved [2111408108/2111408108]



In [ ]:
!unzip -q Negative_tensors.zip

In [ ]:
# !pip install torchvision

In [ ]:
import torchvision.models as models
from PIL import Image
import pandas
from torchvision import transforms
import torch.nn as nn
import time
import torch
import matplotlib.pylab as plt
import numpy as np
from torch.utils.data import Dataset, DataLoader
import h5py
import os
import glob
from matplotlib.pyplot import imshow
torch.manual_seed(0)

In [ ]:
# Create your own dataset object

class Dataset(Dataset):

    # Constructor
    def __init__(self,transform=None,train=True):
        directory="/content/"
        positive="Positive_tensors"
        negative='Negative_tensors'

        positive_file_path=os.path.join(directory,positive)
        negative_file_path=os.path.join(directory,negative)
        positive_files=[os.path.join(positive_file_path,file) for file in os.listdir(positive_file_path) if file.endswith(".pt")]
        negative_files=[os.path.join(negative_file_path,file) for file in os.listdir(negative_file_path) if file.endswith(".pt")]
        number_of_samples=len(positive_files)+len(negative_files)
        self.all_files=[None]*number_of_samples
        self.all_files[::2]=positive_files
        self.all_files[1::2]=negative_files
        # The transform is goint to be used on image
        self.transform = transform
        #torch.LongTensor
        self.Y=torch.zeros([number_of_samples]).type(torch.LongTensor)
        self.Y[::2]=1
        self.Y[1::2]=0

        if train:
            self.all_files=self.all_files[0:30000]
            self.Y=self.Y[0:30000]
            self.len=len(self.all_files)
        else:
            self.all_files=self.all_files[30000:]
            self.Y=self.Y[30000:]
            self.len=len(self.all_files)

    # Get the length
    def __len__(self):
        return self.len

    # Getter
    def __getitem__(self, idx):

        image=torch.load(self.all_files[idx])
        y=self.Y[idx]

        # If there is any transform method, apply it onto the image
        if self.transform:
            image = self.transform(image)

        return image, y

In [ ]:
train_dataset = Dataset(train=True)
validation_dataset = Dataset(train=False)
print("done")

done


<b>Prepare a pre-trained resnet18 model :</b>

<b>Etapa 1</b>: Carregar o modelo pré-treinado <code>resnet18</code> Defina o parâmetro <code>pretrained</code> como verdadeiro:

In [ ]:
# Step 1: Carregue o modelo pré-treinado resnet18
model = models.resnet18(pretrained=True)
# Type your code here

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 63.1MB/s]


In [ ]:
# Etapa 2: Definir o parâmetro que não pode ser treinado para o modelo pré-treinado
for param in model.parameters():
  param.requires_grad = False
# Type your code here

<code>resnet18</code> é usado para classificar 1000 objetos diferentes; como resultado, a última camada tem 1.000 saídas. As 512 entradas vêm do fato de que a camada anteriormente oculta possui 512 saídas.

<b>Etapa 3</b>: Substitua a camada de saída <code>model.fc</code> da rede neural por um objeto <code>nn.Linear</code>, para classificar 2 classes diferentes. Para os parâmetros <code>in_features </code> lembre-se que a última camada oculta possui 512 neurônios.

Imprima o modelo para mostrar se você acertou a resposta.<br> <b>(Seu revisor avaliará com base no que você imprimir aqui.)</b>

In [ ]:
model.fc = nn.Linear(512, 2)
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

Nesta questão você treinará seu modelo:

<b>Etapa 1</b>: Crie uma função de critério de entropia cruzada

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam([param for param in model.parameters() if param.requires_grad], lr=0.003)

<b>Etapa 2</b>: Crie um carregador de treinamento e um objeto carregador de validação, o tamanho do lote deve ter 100 amostras cada.

In [ ]:
batch_size = 100
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size)
validation_loader = DataLoader(dataset=validation_dataset, batch_size=batch_size)

In [ ]:
n_epochs=1
loss_list=[]
accuracy_list=[]
N_test=len(validation_dataset)
N_train=len(train_dataset)
start_time = time.time()

for epoch in range(n_epochs):
    for x, y in train_loader:

        model.train()
        #clear gradient
        optimizer.zero_grad()
        #make a prediction
        z = model(x)
        # calculate loss
        loss = criterion(z, y)
        # calculate gradients of parameters
        loss.backward()
        # update parameters
        optimizer.step()
        loss_list.append(loss.data)
    correct=0

    model.eval()

    with torch.no_grad():
        for x_test, y_test in validation_loader:
            z = model(x_test)  # Faz a predição
            _, yhat = torch.max(z.data, 1)  # Obtém a predição
            correct += (yhat == y_test).sum().item()  # Conta as predições corretas

    accuracy = correct / N_test
    accuracy_list.append(accuracy)

In [ ]:
accuracy

In [ ]:
plt.plot(loss_list)
plt.plot(accuracy_list)
plt.xlabel("iteration")
plt.ylabel("loss")
plt.show()

In [ ]:
plt.plot(accuracy_list)
plt.xlabel("iteration")
plt.ylabel("accuracy")
plt.show()

<b>Identifique as primeiras quatro amostras classificadas incorretamente usando os dados de validação:</b>

In [ ]:
model.eval()
incorrect_samples = []
correct = 0

with torch.no_grad():
    for x_test, y_test in validation_loader:
        z = model(x_test)  # Make prediction
        _, yhat = torch.max(z.data, 1)  # Get prediction
        correct += (yhat == y_test).sum().item()  # Count correct predictions

        # Store incorrect samples
        incorrect_indices = (yhat != y_test).nonzero(as_tuple=True)[0]
        for idx in incorrect_indices:
            if len(incorrect_samples) < 4:  # Stores only the first 4
                incorrect_samples.append((x_test[idx], y_test[idx], yhat[idx]))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np  # Import NumPy for transpose

# ... your code for incorrect_samples

fig, axs = plt.subplots(1, 4, figsize=(12, 3))
for i, (img, true_label, predicted_label) in enumerate(incorrect_samples):
  img = img.squeeze()  # Remove extra dimension if exists

  # Check if grayscale or color image
  if len(img.shape) == 2:
    # Grayscale image, use 'gray' cmap
    axs[i].imshow(img.cpu().numpy(), cmap='gray')
  else:
    # Color image, rearrange channels (assuming RGB or BGR) and use default cmap
    img = img.cpu().numpy().transpose((1, 2, 0))  # Move channel dimension to last
    axs[i].imshow(img)

  axs[i].set_title(f"True: {true_label.item()}\nPred: {predicted_label.item()}")
  axs[i].axis('off')

plt.show()

In [ ]:
for i in range(4):
  print(incorrect_samples[i][1], incorrect_samples[i][2])

In [ ]:
incorrect_samples.index